In [1]:
import pennylane as qml
from pennylane import numpy as np

import matplotlib.pyplot as plt

In [4]:
n_w = 4 # number of wires in 

dev = qml.device("default.qubit", wires=n_w)

In [13]:
def harmonic_wave(n, pos):

    # Set up the parameters
    dx = 1.0/n  # grid spacing

    # Set up the difference operator
    D = np.zeros((n**2, n**2))
    for i in range(n):
        for j in range(n):
            k = i*n + j
            if i > 0:
                D[k, (i-1)*n + j] += 1
            if i < n-1:
                D[k, (i+1)*n + j] += 1
            if j > 0:
                D[k, i*n + j-1] += 1
            if j < n-1:
                D[k, i*n + j+1] += 1
            if pos == 1:
                D[k,k] +=4
            else:
                D[k, k] -= 4

    # Return the final solution
    return D

In [20]:
D = harmonic_wave(n_w, False)

H = qml.pauli_decompose(D)

print(H)

  (-4.0) [I0 I1 I2 I3]
+ (0.5) [I0 I1 X2 X3]
+ (0.5) [I0 I1 Y2 Y3]
+ (0.5) [X0 X1 I2 I3]
+ (0.5) [Y0 Y1 I2 I3]
+ (1.0) [I0 I1 I2 X3]
+ (1.0) [I0 X1 I2 I3]


In [27]:
def ansatz(parameters, wires = n_w, reps = 2):
    n = wires - 1
    
    for j in range(reps):
        for idx, element in enumerate(parameters):
            qml.RX(element[0], wires=idx)
            qml.RY(element[1], wires=idx)  

        
        qml.CNOT(wires=[n, 0])

        for i in range(n):
            qml.CNOT(wires=[i, i+1])

In [26]:
@qml.qnode(dev, interface="autograd")
def cost_fn(params):
    ansatz(params)
    return qml.expval(H)

In [23]:
seednum = 99
np.random.seed(seednum)


randw = np.random.uniform(-np.pi, np.pi, [n_w, 2])
print(randw)

init_params = np.array(randw, requires_grad=True)

[[ 1.08245811 -0.07490563]
 [ 2.0451465  -2.94400917]
 [ 1.935535    0.41228641]
 [-1.27157534 -2.84819479]]


In [24]:
print(qml.draw(cost_fn, expansion_strategy="device")(init_params))

0: ──RX(1.08)───RY(-0.07)─╭X─╭●──RX(1.08)──RY(-0.07)───────────────────────╭X─╭●───────┤ ╭<𝓗>
1: ──RX(2.05)───RY(-2.94)─│──╰X─╭●─────────RX(2.05)───RY(-2.94)────────────│──╰X─╭●────┤ ├<𝓗>
2: ──RX(1.94)───RY(0.41)──│─────╰X────────╭●──────────RX(1.94)───RY(0.41)──│─────╰X─╭●─┤ ├<𝓗>
3: ──RX(-1.27)──RY(-2.85)─╰●──────────────╰X──────────RX(-1.27)──RY(-2.85)─╰●───────╰X─┤ ╰<𝓗>


In [30]:
def qfunc(x, y, wires = 4, reps = 2):
    """ This function is showing how you can use barrier to make the circuits look visually nicer
    but for some reason that doesn't work with the function above...
    """
    for r in range(reps):
        for i in range(wires):
            qml.Hadamard(wires=i)
#         qml.Barrier(wires=[x,y], only_visual = True)
        for j in range(wires):
            qml.PauliX(wires=j)
#         qml.Barrier(wires=[x,y], only_visual = True)
        for k in range(wires - 1):
            qml.CNOT(wires=[k, k+1])
        qml.PauliY(wires=1)
        qml.Barrier(wires=[x,y], only_visual = True)
    return qml.expval(qml.PauliZ(0))

In [31]:
dev = qml.device('default.qubit', wires=2)
qnode = qml.QNode(qfunc, dev)
print(qml.draw(qnode)(0, 3))

0: ──H──X─╭●───────╭||──H──X─╭●───────╭||─┤  <Z>
1: ──H──X─╰X─╭●──Y─│────H──X─╰X─╭●──Y─│───┤     
2: ──H──X────╰X─╭●─│────H──X────╰X─╭●─│───┤     
3: ──H──X───────╰X─╰||──H──X───────╰X─╰||─┤     
